<a href="https://colab.research.google.com/github/mgerlach/advent_of_code/blob/main/2021/aoc2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1) number of increments, window length 1 (part 1), window length 3 (part 2)

In [ ]:
wl = 1  # window length, use 3 for part 2
inc = 0 # increments

m = [int(line) for line in open('drive/MyDrive/AoC/2021/input01.txt')]

for i in range(0, len(m) - wl):
  if m[i] < m[i+wl]:
    inc += 1

inc

1722

2) horizontal/depth course

In [ ]:
prog = [line.split() for line in open('drive/MyDrive/AoC/2021/input02.txt')]  # 0=cmd 1=amount
h = 0
d = 0
for instr in prog:
  cmd = instr[0]
  amount = int(instr[1])
  if cmd == 'forward':
    h += amount
  elif cmd == 'up':
    d -= amount
  elif cmd == 'down':
    d += amount

h * d

1698735

In [ ]:
prog = [line.split() for line in open('drive/MyDrive/AoC/2021/input02.txt')]  # 0=cmd 1=amount
h = 0
d = 0
aim = 0 
for instr in prog:
  cmd = instr[0]
  amount = int(instr[1])
  if cmd == 'forward':
    h += amount
    d += aim * amount
  elif cmd == 'up':
    aim -= amount
  elif cmd == 'down':
    aim += amount

h * d


1594785890

3) Most / least common bits in binary numbers represented as equal length strings

In [ ]:
# count values per bit, keep either most common or least common value for each bit
from functools import reduce

ms = [line.strip() for line in open('drive/MyDrive/AoC/2021/input03.txt')]  # ms = measurements, binary 12 char strings
half = len(ms)/2

# count 1s in each bit position
initial_bit_count1s = [0] * len(ms[0])
count1s = reduce(lambda cnt1s, m: [cnt1s[i] + int(m[i]) for i, x in enumerate(cnt1s)], ms, initial_bit_count1s)

most_common_bitvals = "".join(['1' if c > half else '0' for c in count1s])
least_common_bitvals = "".join(['1' if c < half else '0' for c in count1s])  # could also invert most_common_bitvals

int(most_common_bitvals, 2) * int(least_common_bitvals, 2)

4138664

In [ ]:
# count values for first bit, keep lines with first bit either most or least common value,
# repeat with next bit on filtered list, repeat until only one list entry is left
measurements = [line.strip() for line in open('drive/MyDrive/AoC/2021/input03.txt')]  # binary 12 char strings
bits = len(measurements[0])

# find the measurement according to filtermode
# ms - measurements
# filtermode 1 - keep numbers with bit value = most common value of the bit (equally common - keep those with 1)
# filtermode 0 - keep numbers with bit value = least common value of the bit (equally common - keep those with 0)
def find(ms, filtermode):

  def filter(ms, bit):
    half = len(ms)/2
    # count 1s in bit position
    count1s = sum(int(m[bit]) for m in ms)
    # filter according to count1s and filtermode
    filterval = filtermode if count1s > half else (1-filtermode if count1s < half else filtermode)
    return [m for m in ms if int(m[bit]) == filterval]

  def iter(ms, bit):
    filtered = filter(ms, bit)
    return filtered[0] if len(filtered) == 1 else iter(filtered, bit+1)

  return iter(ms, 0)

int(find(measurements, 1), 2) * int(find(measurements, 0), 2)

4273224

4) Bingo (5x5 Boards)


In [ ]:
# Board structure
# 2-dimensional array of pairs (2-elem lists) with left(0) = number, right(1) = marker (1 = marked, 0 = not marked)
# The number/marker pairs are shared between all boards.

from functools import reduce

# Assume numbers drawn between 0 and 99 (incl.), initialize a marker object (a list length 2) for each number,
# containing the number itself (index 0, actually only needed for display as the index of the marker object 
# already represents the number) and a marker (index 1) of value 0 if the number has not been drawn, yet, and
# 1 if the number was drawn. These markers are shared between all the boards, such that setting the marker to 1
# in one of the numbers, the number will appear as marked on all boards containing it.

markers = [[n, 0] for n in range(0, 100)]
boards = []

for i, l in enumerate(open('drive/MyDrive/AoC/2021/input04.txt')):
  if i == 0:
    # The numbers that are drawn appear in the first line of the input
    numbers = [int(s.strip()) for s in l.split(',')]
  elif i % 6 == 1: 
    # A blank line announcing a new 5x5 board, lines 1, 7, 13, etc.
    boards.append([])
  else:
    # By using the numbers on the board def. as index into markers, the marker objects are shared between all the boards
    boards[int((i-1)/6)].append([markers[int(s.strip())] for s in l.split()])

# Some helpers

def mark(n):
  markers[n][1] = 1  # marks number on all boards

def rowbingo(board) -> bool:
  # bool(list) is True if non-empty, i. e. 5 marked numbers in at least 1 row
  return bool([row for row in board if sum(n[1] for n in row) == 5])  

def colbingo(board) -> bool:
  # The reduce counts marked numbers (row[i][1] == 1) in all columns (i=0..4)
  # true if 5 numbers are marked in any column
  return 5 in reduce(lambda counts, row: [c+row[i][1] for i, c in enumerate(counts)], board, [0, 0, 0, 0, 0])

def bingo(board) -> bool:
  return rowbingo(board) or colbingo(board)

def sum_unmarked(board) -> int:
  return sum(sum(n[0] for n in row if n[1] == 0) for row in board)

def p(board):
  for row in board:
    print(row)

# Part 1: First board to win
bingoboards = []
index = 0
while len(bingoboards) == 0:
  mark(numbers[index])
  bingoboards = list(filter(bingo, boards))  # [board for board in boards if bingo(board)]
  if bingoboards:
    print("First board")
    p(bingoboards[0])
    print(sum_unmarked(bingoboards[0]) * numbers[index])
  else:
    index += 1

# Part 2: Last board to win

# Now need to remove boards with bingo until only one left
# (first remove the one found above)
boards.remove(bingoboards[0])
while len(boards) > 1:
  # print(f"index: {index}, number of boards: {len(boards)}")
  mark(numbers[index])
  boards = [board for board in boards if not bingo(board)]  # list(filter(lambda board: not(bingo(board)), boards))
  index += 1

print("Last board")
# Go on until bingo
while index < len(numbers):
  mark(numbers[index])
  if (bingo(boards[0])):
    p(boards[0])
    print(sum_unmarked(boards[0]) * numbers[index])
    index = len(numbers)  # stop
  else:
    index += 1



First board
[[6, 0], [26, 0], [69, 1], [27, 1], [75, 0]]
[[61, 0], [33, 1], [88, 1], [38, 0], [20, 0]]
[[9, 1], [56, 0], [70, 0], [98, 1], [82, 0]]
[[80, 1], [76, 0], [55, 1], [66, 1], [29, 0]]
[[97, 1], [84, 1], [42, 1], [77, 1], [73, 1]]
41503
Last board
[[94, 1], [38, 1], [96, 1], [44, 1], [46, 0]]
[[10, 1], [14, 1], [75, 0], [97, 1], [76, 1]]
[[7, 1], [61, 1], [56, 1], [36, 1], [1, 0]]
[[81, 0], [67, 1], [49, 1], [78, 1], [86, 1]]
[[31, 1], [65, 1], [88, 1], [24, 0], [63, 1]]
3178


5) Overlapping lines

In [ ]:
# Part 1 only horizontal and vertical lines
# Part 2 also 45 degree diagonal lines
import re

grid = {}  # maps points (x, y) to count

# needed for part 2
def sgn(x): 
  return 1 if x > 0 else (-1 if x < 0 else 0)

def inc_point(x, y):
  if (x, y) not in grid:
    grid[(x, y)] = 1
  else:
    grid[(x, y)] += 1

def add_line(x1, y1, x2, y2):
  if x1==x2:
    for y in range(min(y1,y2), max(y1,y2)+1):
      inc_point(x1, y)
  elif y1==y2:
    for x in range(min(x1,x2), max(x1,x2)+1):
      inc_point(x, y1)
  # Part 2 - could also use this as single option without all the ifs (trusting the input) as it would cover horizontal and vertical lines, too
  elif abs(x2-x1)==abs(y2-y1):
    dx = sgn(x2-x1)
    dy = sgn(y2-y1)
    for i in range(0, abs(x2-x1)+1):
      inc_point(x1 + i * dx, y1 + i * dy)

for l in open('drive/MyDrive/AoC/2021/input05.txt'):
  m = re.fullmatch(r"(\d+),(\d+) -> (\d+),(\d+)\n", l)
  add_line(*[int(m.group(i)) for i in [1, 2, 3, 4]])

print(f"number of points where lines overlap: {sum(1 if grid[k] > 1 else 0 for k in grid.keys())}")

number of points where lines overlap: 22083


6) Population growth

In [ ]:
# Part 1 naive brute force
population = [int(timer_str) for timer_str in open('drive/MyDrive/AoC/2021/input06.txt').readline().split(',')]
iterations = 80
for i in range(0, iterations):
  # print(f"{i}: {len(population)}")
  new_timers = [8] * sum(1 if timer == 0 else 0 for timer in population)
  population = [6 if timer == 0 else timer-1 for timer in population]
  population += new_timers

print(f"{iterations}: {len(population)}")


80: 380612


In [ ]:
# Part 2 a bit more intelligent
pop_raw = [int(timer_str) for timer_str in open('drive/MyDrive/AoC/2021/input06.txt').readline().split(',')]
pop_counts = [0,0,0,0,0,0,0,0,0]  # counts for timer values 0-8
for timer in pop_raw:
  pop_counts[timer] += 1
iterations = 256
for i in range(0, iterations):
  # print(f"{i}: {pop_counts} ({sum(pop_counts)})")
  new_counts = pop_counts[1:]  # decrease all from 1-8
  new_counts[6] += pop_counts[0]  # 0s become 6s
  pop_counts = new_counts + [pop_counts[0]]  # 0s produce new 8s

print(f"{iterations}: {pop_counts} ({sum(pop_counts)})")


256: [145229151101, 175734577513, 183417971628, 196504736072, 231271638229, 222902115155, 282121656192, 119339467551, 153645343459] (1710166656900)


7) Find min diff

In [ ]:
positions = [int(s) for s in open('drive/MyDrive/AoC/2021/input07.txt').readline().split(',')]
# input = [16,1,2,0,4,2,7,1,2,14]
print(f"estimation(part 1): {sum(positions)/len(positions)/2}")

def consumption1(target, current):
  return sum(abs(target - curr) for curr in current)

def consumption2(target, current):
  return sum(int(abs(target-curr)*(abs(target-curr)+1)/2) for curr in current)

print("part 1:", min([(i, consumption1(i, positions)) for i in range(0, len(positions))], key=lambda t: t[1]))
print("part 2:", min([(i, consumption2(i, positions)) for i in range(0, len(positions))], key=lambda t: t[1]))
  

SyntaxError: ignored

8) 4-Digit display wiring

In [ ]:
patterns_and_output = [{'patterns': l[0].split(), 'output': l[1].split()} for l in [line.split('|') for line in open('drive/MyDrive/AoC/2021/input08.txt')]]

# segments for digits 0-9
segments = [ 'abcefg', 'cf', 'acdeg', 'acdfg', 'bcdf', 'abdfg', 'abdefg', 'acf', 'abcdefg', 'abcdfg' ]

# 1 unique digits in output

# segment counts by digit
sc = dict((i, len(s)) for i, s in enumerate(segments))

print("part 1:", sum(1 if len(o) in [sc[1], sc[4], sc[7], sc[8]] else 0 for po in patterns_and_output for o in po['output']))


# 2 find wiring from patterns, then translate/sort output to find the digits

# Segment letters for each digit - this is what the output must be translated to!
# 1:     c     f
# 7: a   c     f    
# 4:   b c d   f     
# 5: a b   d   f g
# 2: a   c d e   g
# 3: a   c d   f g
# 9: a b c d   f g
# 0: a b c   e f g 
# 6: a b   d e f g
# 8: a b c d e f g 

from functools import reduce

# output: map letter in input pattern -> segments (a-g)
def decode(patterns) -> dict:

  # sort patterns by number of segments used for the digit,
  # there are always 1 of length 2 (representing "1"), 1/3 ("7"), 1/4 ("4"), 3/5, 3/6, and 1 of length 7 ("8")
  sp = sorted(patterns, key=lambda s: len(s))
  len2_1_cf = sp[0]  # read: pattern of length 2 = digit "1" = segments c and f
  len3_7_acf = sp[1] 
  len4_4_bcdf = sp[2]  
  len5_235 = sp[3:6]  # read: list of patterns of length 5 (there are always 3), representing digits "2", "3", "5" (5 segments each)
  len6_069 = sp[6:9]  # dito for length 6
  len7_8_abcdefg = sp[9]

  # some utilities 

  def s_diff(s1:str, s2:str) -> str:
    return "".join([c for c in s1 if c not in s2])  

  def s_inter(s1:str, s2:str) -> str:
    return "".join([c for c in s1 if c in s2])

  def s_union(s1:str, s2:str) -> str:
    return "".join(set(s1+s2))

  def chars_in(s1:str, s2:str) -> bool:
    return sum(1 if c in s2 else 0 for c in s1) == len(s1)

  # one of many possible ways of deducting the mapping of the segment codes used in output to the correct segments
  
  mapping = {}
  # 1
  seg_a = s_diff(len3_7_acf, len2_1_cf)
  mapping[seg_a] = 'a'
  # 2 dg = "3" / len5 which includes ("1") - "7"
  seg_dg = s_diff([p for p in len5_235 if chars_in(len2_1_cf, p)][0], len3_7_acf)
  # 3 
  seg_bd = s_diff(len4_4_bcdf, len2_1_cf)
  # 4
  seg_d = s_inter(seg_dg, seg_bd)
  mapping[seg_d] = 'd'
  # 5
  seg_g = s_diff(seg_dg, seg_d)
  mapping[seg_g] = 'g'
  # 6
  seg_b = s_diff(seg_bd, seg_d)
  mapping[seg_b] = 'b'
  # 7
  seg_eg = s_diff(len7_8_abcdefg, s_union(len3_7_acf, len4_4_bcdf))
  # 8
  seg_e = s_diff(seg_eg, seg_g)
  mapping[seg_e] = 'e'
  # 9 f = "6" / len 6 which has only one of the segments of "1" (cf) - segments a,b,d,e,g
  seg_f = reduce(lambda p, seg: s_diff(p, seg), 
                 [seg_a, seg_b, seg_d, seg_e, seg_g], 
                 [p for p in len6_069 if len(s_inter(p, len2_1_cf)) == 1][0])
  mapping[seg_f] = 'f'
  # 10 
  seg_c = s_diff(len2_1_cf, seg_f)
  mapping[seg_c] = 'c'
  return mapping

# apply mapping to output to get correct segment codes
def translate(output, mapping):
  return "".join([mapping[o] for o in output])

# sort letters in a string (needed for looking up digits by segments string)
def s_sort(s:str) -> str:
  return "".join(sorted(s))

# digits by segments, it is important that the segment letters are sorted for each digit!
digits = dict((s, d) for d, s in enumerate(segments))

res = 0
for po in patterns_and_output:
  mapping = decode(po['patterns'])
  decoded_output = sum(digits[s_sort(translate(output, mapping))] * pow(10, i) for i, output in enumerate(reversed(po['output'])))
  # print(decoded_output)
  res += decoded_output

print("part 2:", res)

part 1: 539
part 2: 1084606


9) Height map

In [ ]:
heightmap = [[int(c) for c in line.strip()] for line in open('drive/MyDrive/AoC/2021/input09.txt')]

h = len(heightmap)
w = len(heightmap[0])

# in the following x, y are coordinates
# p (point), lp (low point), nb (neighbor) are tuples representing points with 2 coordinates
# so we can write x, y = p meaning x = p[0], y = p[1]
# low_points, nbs (neighbors), b (basin) are sets of tuples (points)

# this should be memoized / cached
def neighbors(p) -> list:
  x, y = p
  return [(xx, yy) 
          for yy in range(y-1, y+2) 
            for xx in range(x-1, x+2) 
              if ((xx == x) ^ (yy == y)) and xx in range(0, w) and yy in range(0, h)]

def height(p) -> int:
  x, y = p
  return heightmap[y][x]

# 1 - low points 

low_points = [
  (x, y) 
  for y in range(0, h)
    for x in range(0, w) 
      if len([p for p in neighbors((x, y)) if height(p) > height((x, y))]) == len(neighbors((x, y)))]

print("part 1:",
  sum(height(p) + 1 for p in low_points))

# part 2 - basins, using mutable sets

def basin(point) -> set:
  def extend_by(p):
    basin.add(p)
    for nb in neighbors(p):
      if nb not in basin and height(nb) in range(height(p), 9):
        extend_by(nb)
  basin = set()
  extend_by(point)
  return basin

basin_sizes = sorted([len(basin(lp)) for lp in low_points], reverse=True)
print("part 2 (mutable):", basin_sizes[0] * basin_sizes[1] * basin_sizes[2])

# part 2 - basins, using functional adding to sets (return new extended set)

from functools import reduce

def fbasin(point) -> set:
  def extend(basin, p) -> set:
    return reduce(lambda ext_basin, nb: extend(ext_basin, nb) if nb not in ext_basin and height(nb) in range(height(p), 9) else ext_basin, 
                  neighbors(p),      # iterable 'looped' over
                  set([*basin, p]))  # initial value, creates a new set (ext_basin), * explodes/unpacks the current basin
  return extend(set(), point)

fbasin_sizes = sorted([len(fbasin(lp)) for lp in low_points], reverse=True)
print("part 2 (functional):", fbasin_sizes[0] * fbasin_sizes[1] * fbasin_sizes[2])


part 1: 456
part 2 (mutable): 1047744
part 2 (functional): 1047744


10) Chunks surrounded by `(), [], {}, <>`


In [ ]:
lines = [l.strip() for l in open('drive/MyDrive/AoC/2021/input10.txt')]
opening = '([{<'
closing = ')]}>'
opened_by = dict([(c, opening[i]) for i, c in enumerate(closing)])
illegal_scores = {
    ')': 3,
    ']': 57,
    '}': 1197,
    '>': 25137
}
# 1 - corrupted lines, find first illegal char, score the line accordingly, sum up all line scores

def find_illegal_char(line):
  stack = []
  for ch in line:
    if (ch in opening):
      stack.append(ch)
    else:  # assume closed
      if not stack:
        return ch
      opened = stack.pop()
      if opened != opened_by[ch]:
        return ch
  return None

print("part 1:", sum(illegal_scores[il] if il is not None else 0 for il in [find_illegal_char(line) for line in lines]))

# 2 - discard corrupted lines, complete the remaining lines and score each line for completion chars

closed_by = dict([(o, closing[i]) for i, o in enumerate(opening)])
completion_scores = dict([(c, i+1) for i, c in enumerate(closing)])

from functools import reduce

def score_completion(line):
  stack = []
  for ch in line:
    if (ch in opening):
      stack.append(ch)
    else:  # assume closed
      if not stack:
        return 0  # illegal
      opened = stack.pop()
      if opened != opened_by[ch]:
        return 0  # illegal
  return reduce(lambda score, o: 5 * score + completion_scores[closed_by[o]], list(reversed(stack)), 0)

scores = [score_completion(line) for line in lines]
filtered_scores = [s for s in scores if s > 0]

print("part 2:", sorted(filtered_scores)[int(len(filtered_scores)/2)])

part 1: 290691
part 2: 2768166558


11) Flashing octopuses

In [ ]:
grid = [[int(c) for c in l.strip()] for l in open('drive/MyDrive/AoC/2021/input11.txt')]
h = len(grid)
w = len(grid[0])

# this should be memoized / cached
def neighbors(p) -> list:
  x, y = p
  return [(xx, yy) 
          for yy in range(y-1, y+2) 
            for xx in range(x-1, x+2) 
              if (xx != x or yy != y) and xx in range(0, w) and yy in range(0, h)]

def get_energy(p) -> int:
  x, y = p
  return grid[y][x]

# mutates grid
def set_energy(p, energy):
  x, y = p
  grid[y][x] = energy

# increases point p in grid by 1 (mod 10) and returns True if p's energy is now 0 (flashing)
def inc_energy(p):
  x, y = p
  set_energy(p, (get_energy(p) + 1) % 10)
  return get_energy(p) == 0

def p():
  for row in grid:
    print("".join([str(x) for x in row]))

# returns number of flashes in step
def step():
  flashing = [(x, y) for y in range(0, h) for x in range(0, w) if inc_energy((x, y))]
  flashed = flashing
  flashes = len(flashing)
  while flashing:
    increasing = [nb for p in flashing for nb in neighbors(p) if nb not in flashed]
    flashing = [p for p in increasing if get_energy(p) > 0 and inc_energy(p)]
    # p()
    flashed += flashing
    flashes += len(flashing)
  return flashes

# part 1 - how many flashes in 100 steps
flashes = 0
for i in range(0, 100):
#  print(i+1)
  flashes += step() 

print("part 1")
p()
print(flashes)

# part 2 - when do all octopuses flash simultaneously
steps = 100  # from part 1
while sum(sum(row) for row in grid) > 0:
  steps += 1
  step()

print("part 2:")
p()
print(steps)

part 1
1200211116
1239811111
1116581111
1775581111
1754651111
5545861111
8457082111
6790003122
5569003240
5557332200
1673
part 2:
0000000000
0000000000
0000000000
0000000000
0000000000
0000000000
0000000000
0000000000
0000000000
0000000000
279


In [ ]:
# functional

# this should be memoized / cached
def neighbors(grid, p) -> list:
  h = len(grid)
  w = len(grid[0])
  x, y = p
  return [(xx, yy) 
          for yy in range(y-1, y+2) 
            for xx in range(x-1, x+2) 
              if (xx != x or yy != y) and xx in range(0, w) and yy in range(0, h)]

def energy(grid, p):
  x, y = p
  return grid[y][x]

def p(grid):
  for row in grid:
    print("".join([str(e) for e in row]))

# TODO Something does not match the (working) non-functional approach above. Too many flashes in part 1, endless iteration in part 2.

def inc(grid, points, inc_0s=False):
  new_grid = [[(energy + 1) % 10 if (x, y) in points and (inc_0s or energy > 0) else energy for x, energy in enumerate(row)] for y, row in enumerate(grid)]
  new_0s = [p for p in points if energy(new_grid, p) == 0]
  return (new_grid, new_0s)

# returns: new grid, num flashes
def step(gf):
  grid, flashes = gf
  h = len(grid)
  w = len(grid[0])
  increasing = [(x, y) for y in range(0, h) for x in range(0, w)]  # initially, increase energy for all points
  grid, flashing = inc(grid, increasing, inc_0s=True)
  flashed = flashing
  flashes += len(flashing)
  while flashing:
    increasing = [nb for p in flashing for nb in neighbors(grid, p) if nb not in flashed]
    grid, flashing = inc(grid, increasing, inc_0s=False)
    flashed += flashing
    flashes += len(flashing)
  return (grid, flashes)

# part 1 - how many flashes in 100 steps
initial_grid = [[int(c) for c in l.strip()] for l in open('drive/MyDrive/AoC/2021/input11_example.txt')]

from functools import reduce

g, f = reduce(lambda gf, i: step(gf), range(0, 100), (initial_grid, 0))

print('part 1:')
p(g)
print(f)

# part 2 - when do all octopuses flash simultaneously
steps = 100  # from part 1
while sum(sum(row) for row in g) > 0:
  steps += 1
  g, f = step((g, f))

print("part 2:")
p(g)
print(steps)

12) Caves graph

In [ ]:
import re
from functools import reduce

# graph = dict(node -> adjacent nodes) (nodes = caves)

# adds an edge to a graph by merging adjacent nodes for both nodes of given edge 
def add_edge(adj:dict, edge:tuple) -> dict:
  n1, n2 = edge
  return {**adj, **{n1: adj.get(n1, []) + [n2], n2: adj.get(n2, []) + [n1]}}  # functional merge of two dicts

graph = reduce(add_edge, 
               [re.fullmatch('([a-zA-Z]+)-([a-zA-Z]+)', line.strip()).groups() \
                for line in open('drive/MyDrive/AoC/2021/input12.txt')], 
               {})

def is_small(node):
  return node[0].islower()

def is_large(node):
  return node[0].isupper()

def f(node, path):
  # if node == 'end': print(path+[node])
  return 1 if node=='end' else sum(walk1(n, path+[node]) for n in graph[node] if is_large(n) or n not in path)

print('part 1:', walk1('start', []))

def walk2(node, small_twice, path):
  # if node == 'end': print(path+[node])
  return 1 if node=='end' \
           else sum(walk2(n, n if is_small(n) and n in path else small_twice, path+[node]) \
                    for n in graph[node] \
                    if is_large(n) or n not in path or (small_twice is None and n != 'start'))

print('part 2:', walk2('start', None, []))

part 1: 4186
part 2: 92111


13) Folding transparent paper

In [27]:
import re
from functools import reduce

dots = set([tuple([int(s) for s in line.strip().split(',')]) for line in open('drive/MyDrive/AoC/2021/input13.txt') if re.match('^\d', line)])
folds = [re.fullmatch('fold along ([xy])=(\d+)', line.strip()).groups() for line in open('drive/MyDrive/AoC/2021/input13.txt') if line.startswith('fold')]
print("dots :", len(dots))
print("folds:", folds)

def fold(dots, f):
  axis, s = f
  v = int(s)
  return set([(x, y) if x < v else (2*v-x, y) for x, y in dots]) if axis == 'x' else set([(x, y) if y < v else (x, 2*v-y) for x, y in dots])

print('part 1:', len(fold(dots, folds[0])))

def p(code):
  code_size = reduce(lambda m, dot: (max(dot[0], m[0]), max(dot[1], m[1])), code, (0, 0))
  print("code size:", code_size)
  for y in range(0, code_size[1]+1):
    print("".join(['#' if (x, y) in code else ' ' for x in range(0, code_size[0]+1)]))

code = reduce(fold, folds, dots)

print('part 2:')
p(code)

dots : 903
folds: [('x', '655'), ('y', '447'), ('x', '327'), ('y', '223'), ('x', '163'), ('y', '111'), ('x', '81'), ('y', '55'), ('x', '40'), ('y', '27'), ('y', '13'), ('y', '6')]
part 1: 753
part 2:
code size: (38, 5)
#  # #### #    #### #  #   ## ###  #  #
#  #    # #    #    #  #    # #  # # # 
####   #  #    ###  ####    # #  # ##  
#  #  #   #    #    #  #    # ###  # # 
#  # #    #    #    #  # #  # # #  # # 
#  # #### #### #### #  #  ##  #  # #  #
